<a href="https://colab.research.google.com/github/Tinynja/Sarsa-phi-EB/blob/main/rl_paper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import sys

if 'google.colab' in sys.modules:
    !rm -rf *
    !git clone https://github.com/Tinynja/Sarsa-phi-EB
    !mv Sarsa-phi-EB/* .
    !rm -rf Sarsa-phi-EB
    # DON'T install packages defined in Pipfile_colab_remove
    !sed -ri "/$(tr '\n' '|' < Pipfile_Colab_exclude)/d" Pipfile
else:
    print('Skipping GitHub cloning since not running in Colab.')
    
# Install required dependencies
import os

if 'google.colab' in sys.modules:
    # Colab doesn't support pipenv, hence we convert Pipfile into requirements.txt
    if 'requirements_Colab.txt' not in os.listdir():
        !pip install pipenv
        !pipenv lock -r > requirements.txt
    !pip install -r requirements_Colab.txt 1> /dev/null
else:
    !pipenv install 1> /dev/null

# Import all supported ROMs into ALE
!ale-import-roms ROMS
#### ALE-related imports ####
import torch

# Built-in libraries
import re
import sys
import base64
import pickle
import random
import subprocess
from pathlib import Path

# Pypi libraries
import numpy as np
from IPython import display as ipythondisplay
from ale_py import ALEInterface, SDL_SUPPORT
import ale_py.roms as ROMS


!pip install matplotlib numpy gym

# env
import gym

# data manipulation, colab dispaly, and plotting
import pandas as pd
import matplotlib.pyplot as plt


# misc util
import random, glob, base64, itertools
from pathlib import Path
from pprint import pprint

# Configuration
device = 'cuda' if torch.cuda.device_count() else 'cpu'

Cloning into 'Sarsa-phi-EB'...
remote: Enumerating objects: 281, done.
remote: Counting objects: 100% (281/281), done.
remote: Compressing objects: 100% (233/233), done.
remote: Total 281 (delta 79), reused 199 (delta 36), pack-reused 0
Receiving objects: 100% (281/281), 710.79 KiB | 2.78 MiB/s, done.
Resolving deltas: 100% (79/79), done.
/bin/bash: Pipfile_colab_remove: No such file or directory
sed: -e expression #1, char 0: no previous regular expression
Pipfile.lock not found, creating...
Locking [dev-packages] dependencies...
Locking [packages] dependencies...
Building requirements...
Resolving dependencies...
✔ Success! 
Updated Pipfile.lock (03135b)!
[SUPPORTED]                ms_pacman    ROMS/Ms. Pac-Man (1983).bin
[SUPPORTED]                   amidar         ROMS/Amidar (1982).bin
[SUPPORTED]            haunted_house ROMS/Haunted House (Mystery Mansion, Graves' Manor, Nightmare Manor) (1982).bin
[SUPPORTED]                   gopher ROMS/Gopher (Gopher Attack) (1982).bin
[SUPP

In [6]:
class features:
    @staticmethod
    def basic(frame, palette, background, crop_size=torch.Tensor([15,10])):
        # For each color in palette, tell if each pixel is that color
        # e.g. 4x4x3 image, with 2x3 palette, returns 4x4x2
        colors_in_pixels = ((frame-background).unsqueeze(-2) == palette).all(-1)
        # Split the image into `n_subimages`, each with dimension `crop_size`
        frame_dims = torch.Tensor([*frame.shape[:2]])
        n_subimages = (frame_dims/crop_size).prod().item()
        if n_subimages.is_integer():
            n_subimages = int(n_subimages)
        else:
            raise TypeError(f'n_subimages must be an integer, got `{n_subimages}` instead')
        cropped_colors_in_pixels = colors_in_pixels.reshape((n_subimages, *crop_size.int().tolist(), colors_in_pixels.shape[-1]))
        # Apply logical or insize each cropped image
        cropped_features = cropped_colors_in_pixels.any(2).any(1)
        # Flatten the features
        features = cropped_features.flatten()
        return features
    
    @staticmethod
    def b_pros(frame, palette, crop_size=torch.Tensor([15,10])):
        basic_features = features.basic(frame, palette, crop_size=crop_size)
        pass

In [7]:
class EnvALE:
    def __init__(self, rom, out_dir='ale-results', display=False, seed=0, feature_type='ScreenRGB',
                 regen_bg=False, bg_samples=18000):
        self.rom = rom
        self.rom_name = rom.stem
        self.out_dir = Path(out_dir).resolve()
        self.out_dir.mkdir(exist_ok=True)
        self.feature_type = feature_type

        self.ale = ALEInterface()
        self.ale.setInt("random_seed", seed)
        if display and SDL_SUPPORT and 'google.colab' not in sys.modules:
            ale.setBool("sound", True)
            ale.setBool("display_screen", True)
        self.ale.loadROM(rom)

        self.action_space = self.ale.getMinimalActionSet()
        self.color_palette = self._get_color_palette().to(device)

        self.bg_path = Path(f'./backgrounds/{self.rom_name}.pickle')
        if regen_bg or not self.bg_path.exists() or not self.bg_path.is_file():
            self.background = self._get_background(n_samples=bg_samples)
        else:
            with open(self.bg_path, 'rb') as file:
                self.background = pickle.load(file).to(device)
        
        self._set_observe_method(feature_type)

        # Default values
        self._timestep = 0
        self._do_record = False
        self._record_padding = None

    def reset(self, do_record=False):
        self.ale.reset_game()
        observation = self._observe()
        self._timestep = 0

        self._do_record = do_record
        self._handle_recording()
        
        return observation
        
    def step(self, action):
        if isinstance(action, int):
            action = self.action_space[action]
        reward = self.ale.act(action)
        observation = self._observe()
        done = self.ale.game_over()
        self._timestep += 1
        
        self._handle_recording()
        
        return observation, reward, done, None

    def show_video(self, scale=1):
        """Show a .mp4 video in html format of the recorded episode"""
        filepath = self.out_dir.joinpath('record.mp4')
        video_b64 = base64.b64encode(filepath.read_bytes())
        html = f'''<video alt="{filepath}" autoplay loop controls style="height:300px">
                        <source src="data:video/mp4;base64,{video_b64.decode('ascii')}" type="video/mp4" />
                   </video>'''
        ipythondisplay.display(ipythondisplay.HTML(data=html))

    def _set_observe_method(self, feature_type):
        if feature_type == 'ScreenRGB':
            self._observe = lambda: torch.from_numpy(self.ale.getScreenRGB()).to(device)
        elif feature_type == 'ScreenGrayscale':
            self._observe = lambda: torch.from_numpy(self.ale.getScreenGrayscale()).to(device)
        elif feature_type == 'Basic':
            self._observe = lambda: features.basic(frame=torch.from_numpy(self.ale.getScreenRGB()).to(device),
                                                   palette=self.color_palette,
                                                   background=self.background)
        else:
            raise NotImplementedError(f'Feature type `{feature_type}` is not supported')
        
    def _observe(self):
        raise NotImplementedError()
    
    def _get_color_palette(self):
        result = subprocess.run(['python', '-c', f'__import__("ale_py").ALEInterface().loadROM("{str(self.rom)}")'], capture_output=True)
        palette_name = result.stderr.decode().splitlines()[6].strip().split()[-1]
        with open(f'palettes/{palette_name}_Palette.pickle', 'rb') as file:
            palette = pickle.load(file)
        return palette
    
    def _get_background(self, n_samples):
        bg_feature_type = 'ScreenRGB' if self.feature_type not in ['ScreenGrayscale',] else 'ScreenGrayscale'
        self._set_observe_method(bg_feature_type)
        
        sample_i = 0
        pixel_histogram = torch.zeros((*self.ale.getScreenDims(), self.color_palette.shape[0]), dtype=int).to(device)
        while sample_i < n_samples:
            done, observation = False, self.reset()
            while not done and sample_i < n_samples:
                if not sample_i%10:
                    print(f'\rGenerating background... {sample_i}/{n_samples} samples ({sample_i/n_samples:.0%})', end='')
                action = random.choice(self.action_space)
                observation, reward, done, info = self.step(action)
                observation = torch.from_numpy(observation).to(device)
                colors_in_pixels = (observation.unsqueeze(-2) == self.color_palette).all(-1)
                # for i in range(colors_in_pixels.shape[-1]):
                #     print(colors_in_pixels.reshape(-1, 128))
                pixel_histogram += colors_in_pixels
                sample_i += 1
        background_ids = pixel_histogram.argmax(axis=-1)
        background = self.color_palette[background_ids]
        
        self.bg_path.parent.mkdir(exist_ok=True)
        with open(self.bg_path, 'wb') as file:
            pickle.dump(background.cpu(), file)
        
        return background
    
    def _handle_recording(self):
        # Do nothing if not asked to record
        if not self._do_record: return
        # This is a new episode, delete previously recorded steps
        if not self._timestep:
            self.out_dir.joinpath('record').mkdir(exist_ok=True)
            for step_png in self.out_dir.glob('record/step_*.png'):
                step_png.unlink()
            self._record_padding = None
        # Record current timestep png
        out_path = self.out_dir.joinpath(f'record/step_{self._timestep}.png')
        self.ale.saveScreenPNG(str(out_path))
        # Once the episode is over, format all png filenames to have the same integer 0 padding
        if self.ale.game_over():
            self._record_padding = len(str(self._timestep))
            self._standardize_record_padding()
            self._png_to_mp4()
    
    def _standardize_record_padding(self):
        number_pattern = re.compile('\d+')
        for png in self.out_dir.glob('record/step_*.png'):
            timestep = int(number_pattern.search(png.stem).group(0))
            new_name = png.parent.joinpath(f'step_{timestep:0{self._record_padding}d}.png')
            png.rename(new_name)

    def _png_to_mp4(self):
        """Convert the recorded set of png files into a mp4 video"""
        in_dir = self.out_dir.joinpath('record')
        in_pattern = self.out_dir.joinpath(f'record/step_%0{self._record_padding}d.png')
        out_file = self.out_dir.joinpath('record.mp4')
        !cd $in_dir; ffmpeg -hide_banner -loglevel error -r 60 -i $in_pattern -vcodec libx264 -crf 25 -pix_fmt yuv420p -y $out_file


sarsa phi eb

In [8]:
#######################################################################
# Copyright (C)                                                       #
# 2017-2018 Shangtong Zhang(zhangshangtong.cpp@gmail.com)             #
# Permission given to modify the code as long as you keep this        #
# declaration at the top                                              #
#######################################################################

import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from math import floor
from tqdm import tqdm


# all possible actions
ACTIONS = range(4)

# discount is always 1.0 in these experiments
DISCOUNT = 1.0

# use optimistic initial value, so it's ok to set epsilon to 0
EPSILON = 0.05

# maximum steps per episode
STEP_LIMIT = 5000


# get action at @position and @velocity based on epsilon greedy policy and @valueFunction  #########################    use our own get_action. modified it, may work as intended
def get_action(observation, valueFunction):
    if np.random.binomial(1, EPSILON) == 1:
        return np.random.choice(ACTIONS)
    values = []
    for action in ACTIONS:
        values.append(valueFunction.value(observation))  
    return np.argmax(values)



# replacing trace update rule
# @trace: old trace (will be modified)
# @activeTiles: current active tile indices
# @lam: lambda
# @return: new trace for convenience
def replacing_trace(trace, activeTiles, lam):
    active = np.in1d(np.arange(len(trace)), activeTiles)
    trace[active] = 1
    trace[~active] *= lam * DISCOUNT
    return trace



# wrapper class for Sarsa(lambda)
class Sarsa:
    # In this example I use the tiling software instead of implementing standard tiling by myself
    # One important thing is that tiling is only a map from (state, action) to a series of indices
    # It doesn't matter whether the indices have meaning, only if this map satisfy some property
    # View the following webpage for more information
    # http://incompleteideas.net/sutton/tiles/tiles3.html
    # @maxSize: the maximum # of indices
    #the hashing is a lfa?
    def __init__(self, step_size, lam, trace_update=replacing_trace, max_size=28672):
        self.max_size = max_size
        self.trace_update = trace_update
        self.lam = lam

        # divide step size equally to each tiling
        self.step_size = step_size/10

        # weight for each tile
        self.weights = np.zeros(max_size) #max size is the number of features?

        # trace for each tile
        self.trace = np.zeros(max_size)



    # estimate the value of given state and action
    def value(self, observation):
        active_tiles = np.nonzero(observation)
        return np.sum(self.weights[active_tiles])

    # learn with given state, action and target
    def learn(self, observation, target):
        active_tiles = np.nonzero(observation)
        estimation = np.sum(self.weights[active_tiles])
        delta = target - estimation
        print('target: ' + str(target))
        #print('estimation array: ' + str(self.weights[active_tiles]))
        print('estimation: ' + str(np.sum(self.weights[active_tiles])))
        if self.trace_update == replacing_trace:
            self.trace_update(self.trace, active_tiles, self.lam)
        else:
            raise Exception('Unexpected Trace Type')
        self.weights += self.step_size * delta * self.trace
        print('delta: ' + str(delta))
        print('traces: ' + str(self.trace))
        print('weights: ' +  str(self.weights))


# play Mountain Car for one episode based on given method @evaluator
# @return: total steps in this episode
def play(evaluator, env):

    action = random.choice(ACTIONS)
    steps = 0
    while True:
        next_observation, reward, done, info = env.step(action)
        next_action = get_action(next_observation, evaluator)    #########################    use our own get_action  ??? modified it, may work as intented
        steps += 1
        target = reward + DISCOUNT * evaluator.value(next_observation)          ############# use our own value function ??? modified it, may work as intented
        evaluator.learn(observation, target)
        observation = next_observation
        action = next_action
        if done:
            break
        if steps >= STEP_LIMIT:
            print('Step Limit Exceeded!')
            break
    return steps

In [17]:
class BaseAgent:
  """ The base agent class function.
  """
  def __init__(self, nb_features=28672):
    #nothing for now
    self.gamma = 1
    self.features = nb_features
    self.rhos = np.ones(self.features) #stores the rho_i values


  def takeAction(self, t):
    phis = [[0,1,0],[0,1,0],[0,1,0],[1,0,1]]
    return phis[t]


  def updateRho_i(self, counts, t):
    M = self.features
    self.rhos = (counts+1.5)/(t+1)
    print('rhos: ' + str(self.rhos))
    return 0


  def PHI_EB(self, evaluator, env, beta=0.05, t_end=200):
    t = 0
    M = self.features #number of features
    counts = np.zeros(M)
    states = np.zeros((t_end,M)) #stores the previous phis for all timesteps

    action = 1 #starting the game for the agent on the first game
    old_phi = env._observe()
    print('starting iterations')
    print('rhos: ' + str(self.rhos))
    while t < t_end:
      #observe phi(s), reward
      phi, reward, done, info = env.step(action)
      next_action = get_action(phi, evaluator)
      print('phi: ' + str(phi))
      
      #compute rho_t(phi) (feature visit-density)
      if t > 0:
        counts = np.sum(np.where(phi == states[0:t],1,0),axis=0)
        print(counts)
        self.rhos = (counts+0.5)/(t+1)
        print('rhos: ' + str(self.rhos))
        rho_t = np.prod(self.rhos)
      else:
        rho_t = 0.5**M
      print('rho_t ' + str(rho_t))
      #update all rho_i with observed phi
      states[t] = phi
      self.updateRho_i(counts, t+1)
      print('min rho_i_t: ' + str(min(self.rhos)))
      
      #compute rho_t+1(phi)
      new_rho_t = 1
      for i in range(M):
        new_rho_t = new_rho_t*self.rhos[i]
      if new_rho_t <= 1e-323: #this is to avoid division by zero, might need to be tweaked
        new_rho_t = 1e-323
      print('new_rho_t ' + str(new_rho_t))

      #compute Nhat_t(s)
      Nhat_t = rho_t*(1-new_rho_t)/(new_rho_t-rho_t)
      if Nhat_t <= 1e-323: #this is to avoid division by zero again, might need to be tweaked
        Nhat_t = 1e-323

      #compute R(s,a) (empirical reward)
      explorationBonus = beta/np.sqrt(Nhat_t)

      print(explorationBonus)
      reward = reward + explorationBonus

      print('weights: ' + str(evaluator.weights))
      print(max(evaluator.weights))
      print('state value: ' + str(evaluator.value(phi)))
      #pass phi(s) and reward to RL algo to update theta_t
      target = reward + self.gamma * evaluator.value(phi)          ############# use our own value function ??? modified it, may work as intented
      evaluator.learn(old_phi, target)

      if done:
        #break
        env.reset()
        action = 1
        old_phi = env.observe()
      else:
        old_phi = phi
        action = next_action

      t += 1

    return evaluator.weights


In [18]:
from ale_py.roms import Breakout
env = EnvALE(Breakout, feature_type='Basic')
print(env.action_space)
alpha = 0.25
lam = 0.99
evaluator = Sarsa(alpha, lam, replacing_trace,28672)
agent = BaseAgent()
env.reset(do_record=True)
weights = agent.PHI_EB(evaluator, env, beta=0.05, t_end=100)
env.show_video()

[<Action.NOOP: 0>, <Action.FIRE: 1>, <Action.RIGHT: 3>, <Action.LEFT: 4>]
starting iterations
rhos: [1. 1. 1. ... 1. 1. 1.]
phi: tensor([ True, False, False,  ..., False, False, False])
rho_t 0.0
rhos: [0.75 0.75 0.75 ... 0.75 0.75 0.75]
min rho_i_t: 0.75
new_rho_t 1e-323
1.590606226047598e+160
weights: [0. 0. 0. ... 0. 0. 0.]
0.0
state value: 0.0
target: 1.590606226047598e+160
estimation: 0.0
delta: 1.590606226047598e+160
traces: [1. 0. 0. ... 0. 0. 0.]
weights: [3.97651557e+158 0.00000000e+000 0.00000000e+000 ... 0.00000000e+000
 0.00000000e+000 0.00000000e+000]
phi: tensor([ True, False, False,  ..., False, False, False])
0
rhos: 0.25
rho_t 0.25
rhos: 0.5


TypeError: ignored

In [ ]:
#optimisation tests
%%timeit
t = 1
M = 28000
counts = np.zeros(M)
rhos = np.ones(M)
for i in range(M): #M is the number of features of phi
  counts[i] += 1 #since we add phi to the seen states, all the counts are increased by one for t+1
  rhos[i] = (counts[i]+0.5)/(t+1)

10 loops, best of 5: 36.7 ms per loop


In [ ]:
#best one
%%timeit
t = 1
M = 28000
counts = np.zeros(M)
rhos = np.ones(M)
rhos = (counts+1.5)/(t+1)

The slowest run took 12.92 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 5: 54.5 µs per loop


In [ ]:
#compute rho_t(phi) (feature visit-density)
%%timeit
t = 0
t_end = 30
M = 10000
phis = np.zeros((t_end,M))
counts = np.zeros(M)
states = np.zeros((t_end,M))
rhos = np.ones(M)
while t < t_end:
  phi = phis[t]
  if t > 0:
    rho_t = 1
    for i in range(M):
      counts[i] = 0
      for step in range(t):
        if phi[i] == states[step,i]:
          counts[i] += 1
      rhos[i] = (counts[i]+0.5)/(t+1)
      rho_t = rho_t*rhos[i]
  else:
    rho_t = 0.5**M

  #updating rho (other method)
  states[t] = phi
  counts += 1
  rhos = (counts+0.5)/(t+2)
  
  t += 1

1 loop, best of 5: 4.51 s per loop


In [ ]:
#compute rho_t(phi) (feature visit-density)
%%timeit
t = 0
t_end = 5000
M = 1000
phis = np.zeros((t_end,M))
counts = np.zeros(M)
states = np.zeros((t_end,M))
rhos = np.ones(M)
while t < t_end:
  phi = phis[t]
  if t > 0:
    rho_t = 1
    counts = np.zeros(M)
    for step in range(t):
      counts[np.where(phi == states[step])] += 1
    rhos = (counts+0.5)/(t+1)
    rho_t = np.prod(rhos)
  else:
    rho_t = 0.5**M

  #updating rho (other method)
  states[t] = phi
  counts += 1
  rhos = (counts+0.5)/(t+2)
  
  t += 1

1 loop, best of 5: 2min 19s per loop


In [ ]:
#version 3 of optimisation
%%timeit
t = 0
t_end = 5000
M = 1000
#phis = [[0,1,0],[0,1,0],[0,1,0],[1,1,0]]
phis = np.zeros((t_end,M))
counts = np.zeros(M)
states = np.zeros((t_end,M))
rhos = np.ones(M)
while t < t_end:
  phi = phis[t]
  if t > 0:
    rho_t = 1
    counts = np.sum(np.where(phi == states[0:t],1,0),axis=0)
    rhos = (counts+0.5)/(t+1)
    rho_t = np.prod(rhos)
  else:
    rho_t = 0.5**M

  #updating rho (other method)
  states[t] = phi
  counts += 1
  rhos = (counts+0.5)/(t+2)
  
  t += 1

1 loop, best of 5: 45.1 s per loop


In [ ]:
print(phi)
print(states)
print(np.sum(np.where(phi == states,1,0),axis=0))
print(counts[phi == states[3]])

[1, 1, 0]
[[0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 1. 0.]]
[1 4 4]
[0. 0. 0.]
